# Experiment with intermediate answers

In [1]:
from prompting.prompts import INTERMEDIATE_ANSWER_PROMPT_TEMPLATE

In [2]:
question = "Andy's car fuel efficiency is 10 MPG (miles per gallon). If the current price for regular gas is $3/gallon, how much money is Andy's car consuming per week if he only uses his car to go to work from Monday to Friday and the one-way distance between his home and office is 5 miles?"
cot_steps = [
      "Calculate the total distance Andy drives per day: 5 miles (one way) * 2 = 10 miles.",
      "Calculate the total distance Andy drives per week: 10 miles/day * 5 days = 50 miles.",
      "Calculate the number of gallons Andy consumes per week: 50 miles / 10 MPG = 5 gallons.",
      "Calculate the total cost of gas per week: 5 gallons * $3/gallon = $15.",
      "Therefore, Andy's car consumes $15 per week."
    ]

In [3]:
cot_steps_2 = cot_steps[:2]
cot_steps_3 = cot_steps[:3]

In [4]:
filled_prompt_2 = INTERMEDIATE_ANSWER_PROMPT_TEMPLATE.format(
    question=question,
    cot_steps='"' + '",\n\t\t"'.join(cot_steps_2) + '"'
)

In [5]:
print(filled_prompt_2)

Given the question and the reasoning steps, return the correct answer. 
{
    "question": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "cot_steps": [
        "Janet's ducks lay 16 eggs daily.",
        "She uses 3 eggs for breakfast and 4 for muffins, totaling 3 + 4 = 7 eggs.",
        "She sells the remaining eggs, which is 16 - 7 = 9 eggs."
    ],
    "final_answer": "She makes $18 at the farmers' market daily."
},
{
    "question": "A new program had 60 downloads in the first month. The number of downloads in the second month was three times as many as the downloads in the first month, but then reduced by 30% in the third month. How many downloads did the program have total over the three months?",
    "cot_steps": [
        "First, calculate t

In [6]:
filled_prompt_3 = INTERMEDIATE_ANSWER_PROMPT_TEMPLATE.format(
    question=question,
    cot_steps='"' + '",\n\t\t"'.join(cot_steps_3) + '"'
)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteriaList

model_name = "google/gemma-3-1b-it"
tokenizer_generate = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model_generate = AutoModelForCausalLM.from_pretrained(model_name)

C:\Users\libon\anaconda3\envs\bridging_chains\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [30]:
from transformers import StoppingCriteria, StoppingCriteriaList
class StopAfterIntermediateAnswer(StoppingCriteria):
    def __init__(self, tokenizer, stop_strings = ("\n", "\n\n")):
        super().__init__()
        self.stop_strings = [tokenizer.encode(stop_string, add_special_tokens=False)[0] for stop_string in stop_strings]

    def __call__(self, input_ids, scores, **kwargs):
        return input_ids[0, -1] in self.stop_strings

In [31]:
stop_criteria = StoppingCriteriaList([StopAfterIntermediateAnswer(tokenizer_generate)])

In [32]:
tokenized_2 = tokenizer_generate(filled_prompt_2, return_tensors="pt")
output_tokenized = model_generate.generate(**tokenized_2, max_new_tokens=200, do_sample=True, stopping_criteria=stop_criteria)
output = tokenizer_generate.decode(output_tokenized[0])
print(output)

<bos>Given the question and the reasoning steps, return the correct answer. 
{
    "question": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "cot_steps": [
        "Janet's ducks lay 16 eggs daily.",
        "She uses 3 eggs for breakfast and 4 for muffins, totaling 3 + 4 = 7 eggs.",
        "She sells the remaining eggs, which is 16 - 7 = 9 eggs."
    ],
    "final_answer": "She makes $18 at the farmers' market daily."
},
{
    "question": "A new program had 60 downloads in the first month. The number of downloads in the second month was three times as many as the downloads in the first month, but then reduced by 30% in the third month. How many downloads did the program have total over the three months?",
    "cot_steps": [
        "First, calcul

In [33]:
tokenized_3 = tokenizer_generate(filled_prompt_3, return_tensors="pt")
output_tokenized_3 = model_generate.generate(**tokenized_3, max_new_tokens=200, do_sample=True, stopping_criteria=stop_criteria)
output3 = tokenizer_generate.decode(output_tokenized_3[0])
print(output3)

<bos>Given the question and the reasoning steps, return the correct answer. 
{
    "question": "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "cot_steps": [
        "Janet's ducks lay 16 eggs daily.",
        "She uses 3 eggs for breakfast and 4 for muffins, totaling 3 + 4 = 7 eggs.",
        "She sells the remaining eggs, which is 16 - 7 = 9 eggs."
    ],
    "final_answer": "She makes $18 at the farmers' market daily."
},
{
    "question": "A new program had 60 downloads in the first month. The number of downloads in the second month was three times as many as the downloads in the first month, but then reduced by 30% in the third month. How many downloads did the program have total over the three months?",
    "cot_steps": [
        "First, calcul

In [36]:
# Extract final answer
splitted_output = output.split('"final_answer": ')[-1].replace('\n', '').replace('"', '')
splitted_output

'The answer is $15.00'